In [2]:
import pandas as pd
import geopandas as gpd
import rasterio
import requests
import zipfile
import io
import os
import shutil
from shapely.geometry import Point

In [3]:

def load_data_from_url(url, zipped=False, filepath=None, **kwargs):
    """
    Extracts and returns raw data from a URL.

    Parameters:
    url : str
        The URL of the file to be extracted.
    zipped : bool, optional
        If True, the file is expected to be inside a zip file. Default is False.
    filepath : str, optional
        If zipped is True, this is the path to the file inside the archive.
    **kwargs :
        Additional arguments passed to the appropriate reader function 
        (e.g., pd.read_csv, pd.read_excel, gpd.read_file, rasterio.open)

    Returns:
    data : DataFrame, GeoDataFrame, or rasterio.DatasetReader
    """
    def detect_file_type(path):
        ext = os.path.splitext(path.lower())[1]
        if ext in ['.csv']: return 'csv'
        if ext in ['.txt']: return 'txt'
        elif ext in ['.xls']: return 'xls'
        elif ext in ['.xlsx']: return 'xlsx'
        elif ext in ['.shp', '.geojson', '.gpkg', '.json', '.kml']: return 'vector'
        elif ext in ['.tif', '.tiff']: return 'raster'
        else: return 'unknown'

    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    if response.status_code != 200:
        response.raise_for_status()

    if not zipped:
        file_type = detect_file_type(url)
        buffer = io.BytesIO(response.content)

        if file_type == 'csv' or file_type == 'txt':
            data = pd.read_csv(buffer, low_memory=False, **kwargs)
        elif file_type == 'xls':
            data = pd.read_excel(buffer, engine='xlrd', **kwargs)
        elif file_type == 'xlsx':
            data = pd.read_excel(buffer, engine='openpyxl', **kwargs)
        elif file_type == 'vector':
            data = gpd.read_file(buffer, **kwargs).to_crs(saws_crs)
        elif file_type == 'raster':
            data = rasterio.open(buffer, **kwargs)
        else: # fallback on csv
            try:
                data = pd.read_csv(buffer, low_memory=False, **kwargs) 
            except:
                raise ValueError(f"Could not download data from this URL. Please check URL and try again.")

    else:
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
            if filepath is None:
                raise ValueError("Must specify 'filepath' within ZIP archive.")
            file_type = detect_file_type(filepath)

            try:
                with zip_file.open(filepath) as file:
                    if file_type == 'csv':
                        data = pd.read_csv(file, low_memory=False, **kwargs)
                    elif file_type == 'xls':
                        data = pd.read_excel(file, engine='xlrd', **kwargs)
                    elif file_type == 'xlsx':
                        data = pd.read_excel(file, engine='openpyxl', **kwargs)
                    elif file_type in ['vector', 'raster']:
                        raise NotImplementedError("Shapefiles and rasters require extraction.")
            except:
                zip_file.extractall('extracted_data')
                full_path = os.path.abspath(os.path.join('extracted_data', filepath))
                if file_type == 'csv':
                    data = pd.read_csv(full_path, low_memory=False, **kwargs)
                elif file_type == 'xls':
                    data = pd.read_excel(full_path, engine='xlrd', **kwargs)
                elif file_type == 'xlsx':
                    data = pd.read_excel(full_path, engine='openpyxl', **kwargs)
                elif file_type == 'vector':
                    data = gpd.read_file(full_path, **kwargs).to_crs(saws_crs)
                elif file_type == 'raster':
                    data = rasterio.open(full_path, **kwargs)
                shutil.rmtree('extracted_data')

    return data


def lat_long_to_point(df, lat_col, long_col):
    
    """
    This function takes a DataFrame with lat/long columns stored as floats
    and converts it into a GeoDataFrame with a point geometry.

    Arguments:
    df : DataFrame
        The DataFrame to be converted.
    lat_col : str
        The name of the column containing latitude values.
    long_col : str
        The name of the column containing longitude values.

    Returns:
    df : GeoDataFrame
        The modified DataFrame as a GeoDataFrame with point geometry.
    """

    df_geo = [Point(lon, lat) for lon, lat in zip(df[long_col], df[lat_col])]
    df = gpd.GeoDataFrame(df, geometry=df_geo, crs = 'EPSG:4326')
    df = df.to_crs(saws_crs)
    
    return df


In [4]:
data_url = 'https://www.sciencebase.gov/catalog/file/get/58937228e4b0fa1e59b73361?f=__disk__5a%2Fae%2F1a%2F5aae1aa25f84b94737628e43ef82e34f6897a63b'

data=load_data_from_url(data_url, zipped=True, filepath='Major_Ions.csv')
print(data.head())


  sourceID  unique_site_ID samp_id  proprietary_data state_alpha fips_cd  \
0    AZDEQ  AZDEQ-az000779     NaN                 0          AZ   04021   
1    AZDEQ  AZDEQ-az001146     NaN                 0          AZ   04013   
2    AZDEQ  AZDEQ-az002056     NaN                 0          AZ   04021   
3    AZDEQ  AZDEQ-az002153     NaN                 0          AZ   04021   
4    AZDEQ  AZDEQ-az006219     NaN                 0          AZ   04013   

  county_nm  dec_lat_va  dec_long_va location_flag  ...  \
0     Pinal   32.938241  -111.936083           2.0  ...   
1  Maricopa   33.005930  -112.676581           2.0  ...   
2     Pinal   32.676461  -111.516639           2.0  ...   
3     Pinal   33.065584  -111.975132           2.0  ...   
4  Maricopa   33.518722  -113.069351           2.0  ...   

   model_pp_Anhydrite_closed_16x  model_pp_Barite_closed_16x  \
0                            0.0                         0.0   
1                            0.0                         0.0

In [5]:
# Only include samples where TDS_mgL is greater than or equal to 1000
filtered_data = data[data['TDS_mgL'] >= 1000]

num_rows_total = len(data)  # total rows before filtering
num_rows_filtered = len(filtered_data)  # rows after filtering
num_rows_excluded = num_rows_total - num_rows_filtered

print("Number of rows in original data:", num_rows_total)
print("Number of rows in filtered data (TDS >= 1000 mg/L):", num_rows_filtered)
print("Number of rows excluded (TDS < 1000 mg/L):", num_rows_excluded)

Number of rows in original data: 123699
Number of rows in filtered data (TDS >= 1000 mg/L): 23454
Number of rows excluded (TDS < 1000 mg/L): 100245


In [6]:
# Create a DataFrame with just the column names as a single column
pd.Series(data.columns, name="Column_Name").to_csv("column_headers_list.csv", index=False)

print("CSV with column headers saved as column_headers.csv")

CSV with column headers saved as column_headers.csv


In [7]:
filtered_data2 = filtered_data[filtered_data['model_pct_err'].abs() <= 10]

num_rows_filtered2 = len(filtered_data2)  # rows after filtering
num_rows_excluded2 = num_rows_filtered - num_rows_filtered2

print("Number of rows in filtered data (model_pct_err.abs() <= 10):", num_rows_filtered2)
print("Number of rows excluded (model_pct_err.abs() >= 10):", num_rows_excluded2)

Number of rows in filtered data (model_pct_err.abs() <= 10): 16706
Number of rows excluded (model_pct_err.abs() >= 10): 6748


In [8]:

required_columns = [
    'well_depth_ft',       # well depth
    'Temp_C',    # temperature in Celsius -> alternative is model_temp_c
    'ph',                # pH measurement -> alternative is model_pH
    'Alk_mgL',   # alkalinity as mg/L CaCO3 -> alternative is model_Alk_eqL (or Alk_mgL_flag)
    'Si_mgL'          # silica in mg/L --> alternative is model_Si_molL (or Si_mgL_flag)
]

# Drop rows where ANY of these columns are missing
filtered_data3 = filtered_data2.dropna(subset=required_columns)

num_rows_filtered3 = len(filtered_data3)  # rows after filtering
num_rows_excluded3 = num_rows_filtered2 - num_rows_filtered3

print("Remaining rows after removing samples with missing key parameters:",
      len(filtered_data3)) 
print("Number of rows excluded due to missing key parameters:", num_rows_excluded3)

Remaining rows after removing samples with missing key parameters: 14943
Number of rows excluded due to missing key parameters: 1763


In [9]:
mask = (
    (filtered_data2['well_depth_ft'].notna() | filtered_data2['well_depth2_ft'].notna()) &
    (filtered_data2['Temp_C'].notna() | filtered_data2['model_temp_c'].notna()) &
    (filtered_data2['ph'].notna() | filtered_data2['model_pH'].notna()) &
    filtered_data2['Alk_mgL'].notna() &
    filtered_data2['Si_mgL'].notna()
)
 
filtered_data3 = filtered_data2[mask]
num_rows_filtered2 = len(filtered_data2)
num_rows_filtered3 = len(filtered_data3)
num_rows_excluded3 = num_rows_filtered2 - num_rows_filtered3

print("Remaining rows after removing samples with missing key parameters:", num_rows_filtered3)
print("Number of rows excluded due to missing key parameters:", num_rows_excluded3)


Remaining rows after removing samples with missing key parameters: 14943
Number of rows excluded due to missing key parameters: 1763


In [10]:
step2=filtered_data2
has_well_depth = step2['well_depth2_ft'].notna() | step2['well_depth_ft'].notna()

# Temp/pH/Alk/Silica: require measured values (modelling fields not used)
has_temp = step2['Temp_C'].notna()
has_pH = step2['ph'].notna()
has_alk = step2['Alk_mgL'].notna()
has_silica = step2['Si_mgL'].notna()

mask = has_well_depth & has_temp & has_pH & has_alk & has_silica
step3 = step2[mask]

print("After charge balance filter:", len(step2))
print("After missing-key-parameter filter:", len(step3),
      "-> excluded:", len(step2) - len(step3))

diff = step2.loc[~mask]  # These are the excluded rows
print(diff[['well_depth_ft','well_depth2_ft',
            'Temp_C','ph','Alk_mgL','Si_mgL',
            'Alk_mgL_flag','Si_mgL_flag']])

After charge balance filter: 16706
After missing-key-parameter filter: 14943 -> excluded: 1763
        well_depth_ft  well_depth2_ft  Temp_C    ph  Alk_mgL  Si_mgL  \
1               744.0             NaN   27.98  7.70     98.0     NaN   
3               625.0             NaN   23.65  7.47    260.0     NaN   
9               904.0             NaN   22.09  7.82    190.0     NaN   
11             1000.0             NaN   27.80  8.04    210.0     NaN   
12             1200.0             NaN   20.40  7.35    250.0     NaN   
...               ...             ...     ...   ...      ...     ...   
122387          320.0             NaN    7.00  7.80    861.0     NaN   
122423            NaN             NaN    9.00  7.40    670.0    19.0   
122834            NaN             NaN    6.00  4.70      3.0    32.0   
122843          258.0             NaN    8.50  9.10    275.0     NaN   
122865          275.0             NaN    8.00  7.40    407.0     NaN   

       Alk_mgL_flag Si_mgL_flag  
1     

In [11]:
import numpy as np

def is_valid_measurement(value, flag=None):
    """
    Determines if a measurement is valid (not missing), considering flags.
    Flags like '<' or 'ppm' mean censored but valid.
    """
    if pd.notna(value):
        return True
    else:
        # If value missing (NaN), count as invalid
        return False
    
# Well depth: either sample level or site level present
has_well_depth = filtered_data2['well_depth2_ft'].notna() | filtered_data2['well_depth_ft'].notna()

# Temperature, pH: measured values present (ignore modeled)
has_temp = filtered_data2['Temp_C'].notna()
has_pH = filtered_data2['ph'].notna()

# Alkalinity and Silica: treat flagged values as presence
# For Alk_mgL, count as present if measurement is present OR flagged as '<' or 'ppm'

alk_present = (
    filtered_data2['Alk_mgL'].notna() |
    filtered_data2['Alk_mgL_flag'].isin(['<', '< ppm', 'ppm', np.nan])
)
# The np.nan inclusion means if flag missing, count as present since no negative qualifier

silica_present = (
    filtered_data2['Si_mgL'].notna() |
    filtered_data2['Si_mgL_flag'].isin(['<', '< ppm', 'ppm', np.nan])
)
# Combine all conditions for filtering
mask = has_well_depth & has_temp & has_pH & alk_present & silica_present

filtered_data3 = filtered_data2[mask]

num_rows_filtered3 = len(filtered_data3)
num_rows_excluded3 = len(filtered_data2) - num_rows_filtered3

print("Remaining rows after removing samples missing key parameters (USGS style):", num_rows_filtered3)
print("Number of rows excluded due to missing key parameters (USGS style):", num_rows_excluded3)

Remaining rows after removing samples missing key parameters (USGS style): 15700
Number of rows excluded due to missing key parameters (USGS style): 1006


In [12]:
# See unique values in Alk_mgL_flag, with counts
print("Alkalinity flag values:")
print(filtered_data2['Alk_mgL_flag'].value_counts(dropna=False))

print("\nSilica flag values:")
print(filtered_data2['Si_mgL_flag'].value_counts(dropna=False))


Alkalinity flag values:
Alk_mgL_flag
NaN    16687
<         19
Name: count, dtype: int64

Silica flag values:
Si_mgL_flag
NaN    14199
ppm     2484
<         23
Name: count, dtype: int64


In [13]:
mask = (
    (filtered_data2['well_depth_ft'].notna() | filtered_data2['well_depth2_ft'].notna()) &
    (filtered_data2['Temp_C'].notna() | filtered_data2['model_temp_c'].notna()) &
    (filtered_data2['ph'].notna() | filtered_data2['model_pH'].notna()) &
    filtered_data2['Alk_mgL'].notna() &
    filtered_data2['Si_mgL'].notna()
)
 
filtered_data3 = filtered_data2[mask]
num_rows_filtered2 = len(filtered_data2)
num_rows_filtered3 = len(filtered_data3)
num_rows_excluded3 = num_rows_filtered2 - num_rows_filtered3

print("Remaining rows after removing samples with missing key parameters:", num_rows_filtered3)
print("Number of rows excluded due to missing key parameters:", num_rows_excluded3)

print(filtered_data3['Si_mgL'].describe())
print(filtered_data3.sort_values(by='Si_mgL', ascending=False)['Si_mgL'].head(10))



Remaining rows after removing samples with missing key parameters: 14943
Number of rows excluded due to missing key parameters: 1763
count    14943.000000
mean        18.940450
std         18.153428
min          0.000000
25%          9.000000
50%         15.000000
75%         25.000000
max       1000.000000
Name: Si_mgL, dtype: float64
62822     1000.0
2617       843.0
104993     400.0
115836     290.0
108164     244.0
41193      240.0
33255      187.0
66426      180.0
641        178.0
66427      170.0
Name: Si_mgL, dtype: float64


In [14]:
# Get the indices of the two rows with highest silica (Si_mgL)
top_two_indices = filtered_data3.sort_values(by='Si_mgL', ascending=False).head(2).index

# Drop those two rows from DataFrame
filtered_data_final = filtered_data3.drop(top_two_indices)

print("Excluded rows with anomalously high silica values.")
print(f"Remaining rows after exclusion: {len(filtered_data_final)}")

Excluded rows with anomalously high silica values.
Remaining rows after exclusion: 14941


In [23]:
# List of state abbreviations you want to keep
states_to_keep = ["FL", "CA", "TX", "AZ", "NM"]

# Filter DataFrame to only keep those states
data_by_state = filtered_data_final[filtered_data_final['state_alpha'].isin(states_to_keep)]

print(f"Remaining rows after keeping only {states_to_keep}: {len(data_by_state)}")
data_CA = data_by_state[data_by_state['state_alpha'] == 'CA']
print(f"Filtered data for California: {len(data_CA)}")
data_FL = data_by_state[data_by_state['state_alpha'] == 'FL']
print(f"Filtered data for Florida: {len(data_FL)}")
data_TX = data_by_state[data_by_state['state_alpha'] == 'TX']
print(f"Filtered data for Texas: {len(data_TX)}")
data_AZ = data_by_state[data_by_state['state_alpha'] == 'AZ']
print(f"Filtered data for Arizona: {len(data_AZ)}")
data_NM = data_by_state[data_by_state['state_alpha'] == 'NM']
print(f"Filtered data for New Mexico: {len(data_NM)}")

Remaining rows after keeping only ['FL', 'CA', 'TX', 'AZ', 'NM']: 4810
Filtered data for California: 1268
Filtered data for Florida: 386
Filtered data for Texas: 2388
Filtered data for Arizona: 605
Filtered data for New Mexico: 163


In [33]:

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

df = data_by_state.copy()

# === Stage 1: Ion & TDS Processing ===
# Define which columns are cations, anions, etc.
cations = ['Ca_mgL', 'Mg_mgL', 'Na_mgL', 'K_mgL']  # replace with your actual columns
anions = ['Cl_mgL', 'SO4_mgL', 'HCO3_mgl']         # replace with your actual columns
silica_col = 'Si_mgL'
dissolved_solids_col = 'TDS_mgL'

# Molar masses (g/mol) for conversions
molar_masses = {
    'Ca_mgL': 40.078,
    'Mg_mgL': 24.305,
    'Na_mgL': 22.990,
    'K_mgL': 39.098,
    'Cl_mgL': 35.45,
    'SO4_mgL': 96.06,  # SO4-2
    'HCO3_mgl': 61.0168,
    silica_col: 60.08  # SiO2
}

# Charges for equivalent calculations
charges = {
    'Ca_mgL': 2,
    'Mg_mgL': 2,
    'Na_mgL': 1,
    'K_mgL': 1,
    'Cl_mgL': 1,
    'SO4_mgL': 2,
    'HCO3_mgl': 1
}

# Convert mg/L to equivalents for major ions
for ion in cations + anions:
    eq_col = ion + '_eq'
    df[eq_col] = (df[ion] / molar_masses[ion]) * charges[ion]

# Totals
df['total_cations_eq'] = df[[c + '_eq' for c in cations]].sum(axis=1)
df['total_anions_eq']  = df[[a + '_eq' for a in anions]].sum(axis=1)

# Fractions
for ion in cations:
    df[ion + '_frac'] = df[ion + '_eq'] / df['total_cations_eq']
for ion in anions:
    df[ion + '_frac'] = df[ion + '_eq'] / df['total_anions_eq']

# Print just the new columns created
#new_cols = [ion + '_eq' for ion in cations + anions] + ['total_cations_eq', 'total_anions_eq']
#print(df[new_cols].head())

# Silica fraction
df['Si_molL'] = df[silica_col] / molar_masses[silica_col]
df['total_ion_molL'] = (
    df[[ion + '_eq' for ion in cations + anions]]
    .div([charges[i] for i in cations + anions], axis=1)
    .sum(axis=1)
)
df['Si_molL_frac'] = df['Si_molL'] / df['total_ion_molL']

# Log transform TDS
df['TDS_log10'] = np.log10(df[dissolved_solids_col])

# === Stage 2: Data Normalization ===
# Prepare dataframe for clustering
untouched = ['ph', 'Temp_C']
features = [col for col in df.columns if col.endswith('_frac')] + ['TDS_log10'] + untouched
X = df[features]
print("Features used for clustering:", X.columns.tolist())
# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# R² vs k
K_range = range(1, 10)
r2_values = []
tot_ss = np.sum((X_scaled - X_scaled.mean(axis=0)) ** 2)

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X_scaled)
    between_ss = tot_ss - km.inertia_
    r2_values.append(between_ss / tot_ss)

plt.plot(K_range, r2_values, marker='o')
plt.xlabel('k')
plt.ylabel('R²')
plt.title('Elbow method for k-means')
plt.show()

# Final k=4 (example)
k_optimal = 4
km_final = KMeans(n_clusters=k_optimal, random_state=42, n_init=10)
df['Cluster'] = km_final.fit_predict(X_scaled)

# Save results
#df.to_csv("geochem_clusters_with_metadata.csv", index=False)



Features used for clustering: ['Ca_mgL_frac', 'Mg_mgL_frac', 'Na_mgL_frac', 'K_mgL_frac', 'Cl_mgL_frac', 'SO4_mgL_frac', 'HCO3_mgl_frac', 'Si_molL_frac', 'TDS_log10', 'ph', 'Temp_C']


ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values